In [5]:
import urllib
import zipfile

import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Reshape
from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:
#def normalize_series(data, min, max):
#data = data - min
#data = data / max
#  return data

In [7]:
  # Reads the dataset from the CSV.
df = pd.read_csv('household_power_consumption.csv',
                 sep=',',
                 infer_datetime_format=True,
                 index_col='datetime',
                 header=0)
df

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
...,...,...,...,...,...,...,...
2007-02-14 17:19:00,0.636,0.140,241.16,2.6,0.0,0.0,0.0
2007-02-14 17:20:00,0.552,0.000,240.46,2.2,0.0,0.0,0.0
2007-02-14 17:21:00,0.538,0.000,239.74,2.2,0.0,0.0,0.0


In [16]:
# Number of features in the dataset. We use all features as predictors to
# predict all features at future time steps.
N_FEATURES = len(df.columns) # DO NOT CHANGE THIS
N_FEATURES

7

In [17]:
from sklearn.preprocessing import MinMaxScaler
# Normalizes the data
# The main idea behind normalization/standardization is always the same.
#Variables that are measured at different scales do not contribute equally to the model fitting & model
#learned function and might end up creating a bias. Thus, to deal with this potential problem feature-wise normalization such
#as MinMax Scaling is usually used prior to model fitting.

# One Way
data = df.values  ## Return a Numpy representation of the DataFrame.
data = data-data.min(axis=0)
data
data = data/data.max(axis=0)
data

#Another Way
# scaler = MinMaxScaler()
# scaler.fit(data)

# data_scaled = scaler.transform(data)
# data_scaled


array([[0.44304913, 0.47826087, 0.37601776, ..., 0.        , 0.01282051,
        0.85      ],
       [0.56906808, 0.49885584, 0.33123612, ..., 0.        , 0.01282051,
        0.8       ],
       [0.57061027, 0.56979405, 0.31865285, ..., 0.        , 0.02564103,
        0.85      ],
       ...,
       [0.03789381, 0.        , 0.55736491, ..., 0.        , 0.        ,
        0.        ],
       [0.03635162, 0.        , 0.60695781, ..., 0.        , 0.        ,
        0.        ],
       [0.03855475, 0.        , 0.62694301, ..., 0.        , 0.01282051,
        0.        ]])

In [22]:
# Splits the data into training and validation sets.
SPLIT_TIME = int(len(data) * 0.5) # DO NOT CHANGE THIS
x_train = data[:SPLIT_TIME]
x_valid = data[SPLIT_TIME:]

#x_train
#x_valid


array([[0.14937211, 0.        , 0.57142857, ..., 0.        , 0.        ,
        0.9       ],
       [0.20423001, 0.29290618, 0.53478905, ..., 0.        , 0.        ,
        0.85      ],
       [0.25203789, 0.5423341 , 0.54330126, ..., 0.        , 0.        ,
        0.9       ],
       ...,
       [0.03789381, 0.        , 0.55736491, ..., 0.        , 0.        ,
        0.        ],
       [0.03635162, 0.        , 0.60695781, ..., 0.        , 0.        ,
        0.        ],
       [0.03855475, 0.        , 0.62694301, ..., 0.        , 0.01282051,
        0.        ]])

In [28]:

tf.keras.backend.clear_session()
tf.random.set_seed(42)


In [27]:

BATCH_SIZE = 32

In [4]:

N_PAST = 24

In [24]:
# Number of future time steps which are to be predicted.
N_FUTURE = 24  

In [25]:

SHIFT = 1 

In [32]:
# Code to create windowed train and validation datasets.

ds = tf.data.Dataset.from_tensor_slices(x_train)
ds = ds.window(size=N_PAST + N_FUTURE, shift=SHIFT, drop_remainder=True)
for element in ds:
  print(element)
ds = ds.flat_map(lambda w: w.batch(N_PAST + N_FUTURE))
ds = ds.map(lambda w: (w[:N_PAST], w[N_PAST:]))
train_set = ds.batch(BATCH_SIZE).prefetch(1)
train_set

Streaming output truncated to the last 5000 lines.
<_VariantDataset element_spec=TensorSpec(shape=(7,), dtype=tf.float64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(7,), dtype=tf.float64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(7,), dtype=tf.float64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(7,), dtype=tf.float64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(7,), dtype=tf.float64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(7,), dtype=tf.float64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(7,), dtype=tf.float64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(7,), dtype=tf.float64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(7,), dtype=tf.float64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(7,), dtype=tf.float64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(7,), dtype=tf.float64, name=None)>
<_VariantDataset element_spec=Tensor

In [33]:
ds2 = tf.data.Dataset.from_tensor_slices(x_valid)
ds2 = ds2.window(size=N_PAST + N_FUTURE, shift=SHIFT, drop_remainder=True)
ds2 = ds2.flat_map(lambda w: w.batch(N_PAST + N_FUTURE))
ds2 = ds2.map(lambda w: (w[:N_PAST], w[N_PAST:]))
train_set = ds2.batch(BATCH_SIZE).prefetch(1)

In [ ]:
# Code to define your model.
model = tf.keras.models.Sequential([

# ADD YOUR LAYERS HERE.
tf.keras.layers.Bidirectional(
tf.keras.layers.LSTM(64, return_sequences=True, input_shape=[N_PAST, N_FEATURES])),
tf.keras.layers.Dense(7, activation='sigmoid'),
tf.keras.layers.Dense(N_FEATURES),
Reshape([N_FUTURE, -1])
 ])

optimizer =  tf.keras.optimizers.Adam()
model.compile(
      # YOUR CODE HERE
      optimizer=optimizer, loss=tf.keras.losses.Huber(), metrics=['mae']
  )
model.fit(
      # YOUR CODE HERE
      train_set, batch_size=BATCH_SIZE, epochs=20, callbacks=[
          ModelCheckpoint(
              filepath='mymodel.h5',
              monitor='loss',
              mode='min',
              save_best_only=True,
              save_weights_only=False,
              verbose=1
          )
      ])